## 훈련과정 : R·파이썬 기반 빅데이터 분석 전문가 양성과정
- 교과목 평가 : 머신러닝 기반 데이터 분석
 훈련과정 : R·파이썬 기반 빅데이터 분석 전문가 양성과정
- 교과목 평가 : 머신러닝 기반 데이터 분석
- 성명 : 용광순
- 점수 :

#### Q. 아래사항을 준수하여 타이타닉 생존자 예측모델을 개발하세요.

- 데이터 : 
  - 제공 데이터 파일 : titanic3.csv
  - 훈련/검증용 데이터 : 평가 데이터 = 8 : 2
  - 훈련/검증용 데이터로 모델 학습 및 검증하고 평가 데이터는 최종 평가에만 사용


- 모델 개발 방법 :
  - 데이터 전처리 및 탐색적 분석을 통하여 파생변수 최소 2개 이상 개발
  - 알고리즘은 최소한 3개 이상 적용(Decision Tree, Random Forest, Logistic Regression은 필수)


- 훈련 및 평가 방법 :
  - GridSearchCV API를 활용하여 교차검증 및 최적 하이퍼파라미터 찾아서 학습 및 검증 수행

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['age'].fillna(df['age'].mean(), inplace=True)
    df['cabin'].fillna('N', inplace=True)
    df['fare'].fillna(df['fare'].mean(), inplace=True)
    df['embarked'].fillna('N', inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거 (name, ticket, boat, body, home.dest)
def drop_features(df):
    df.drop(['home.dest','boat','body','name','ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩
def format_features(df):
    df['cabin'] = df['cabin'].str[:1]
    features = ['cabin','sex','embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [6]:
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

titanic_df = pd.read_csv('titanic3.csv')
transform_features(titanic_df)

display(titanic_df.head())
print()

df1 = titanic_df.loc[:, ['pclass','sex','age','fare','survived']]
df1.to_pickle('t3_df.pkl')

df1 = pd.read_pickle('t3_df.pkl')

display(df1.head())

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,0,29.00,0,0,211.3375,1,3
1,1,1,1,0.92,1,2,151.5500,2,3
2,1,0,0,2.00,1,2,151.5500,2,3
3,1,0,1,30.00,1,2,151.5500,2,3
4,1,0,0,25.00,1,2,151.5500,2,3


,pclass,sex,age,fare,survived
0,1,0,29.00,211.3375,1
1,1,1,0.92,151.5500,1
2,1,0,2.00,151.5500,0
3,1,1,30.00,151.5500,0
4,1,0,25.00,151.5500,0


In [8]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
X = df1[['pclass', 'sex', 'age', 'fare']]
y = df1['survived']

# 독립변수 정규화(평균 0, 분산 1인 표준정규분포)
X = preprocessing.StandardScaler().fit(X).transform(X)

# 학습용 데이터와 평가용 데이터를 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

print(X_train.shape)
print(X_test.shape)
print()
print(X.mean())
print(X.std())
print(type(y_test))

(1047, 4)
(262, 4)

-1.1670488020505846e-16
1.0
<class 'pandas.core.series.Series'>


In [9]:
# Decision Tree
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier() # 객체 생성
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test) # 검증용 데이터로 예측

print(dt_pred[0:10])
print(y_test.values[0:10])

accuracy = accuracy_score(y_test, dt_pred)
print('dt 예측 정확도: ', accuracy)

[0 0 0 1 0 0 0 1 1 0]
[1 0 0 1 0 0 0 1 1 0]
dt 예측 정확도:  0.7748091603053435


In [10]:
# Random Forest (투표를 해가며 결정해가는 방법)
from sklearn.ensemble import RandomForestClassifier # (분류기)

rf_model = RandomForestClassifier() # 객체 생성
rf_model.fit(X_train, y_train) # 학습을 시킨다. (트레이닝 데이터를 이용)
rf_pred = rf_model.predict(X_test) # 학습시킨 모델을 통해 예측한다. (검증용 데이터로)

rf_accuracy = accuracy_score(y_test, rf_pred) # 예측한 것과 실제 y값으로 비교해서 예측 정확도를 구한다.
print('rf 예측 정확도: ', rf_accuracy)

rf 예측 정확도:  0.8053435114503816


In [11]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

lr_accuracy = accuracy_score(y_test, lr_pred)
print('lr 예측 정확도: ', lr_accuracy)

lr 예측 정확도:  0.7557251908396947


In [12]:
# 독립변수, 종속변수 분리
t_df = pd.read_pickle('t_df.pkl')

y_df = t_df.survived
X_df = t_df.drop('survived', axis=1)

# 학습용 평가용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=11)

# 분류기 객체 생성
# dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
# lr_clf = LogisticRegression(random_state=11)

from sklearn.model_selection import KFold

# KFold 교차 검증 수행()
# K개의 데이터 폴드 세트를 만들어서 K번 만큼 각 폴드 세트에
# 학습과 검증 평가를 반복, K가 5이면 5번 평가를 평균한 결과로 예측 성능 검증
# 예측 성능 평가
def exec_kfold(clf, folds=5):      # clf는 클래시파이어, folds는 k의 개수
    kfold = KFold(n_splits=folds)  # n_splits = folds
    scores = []                    # 만들어서 저장하는 공간( foltset 설정하는 단계 )
    
    for iter_count, (train_index, test_index) \
    in enumerate(kfold.split(X_df)):            # enumerate 해서 독립변수 쪽의 5개를 하나씩 작업해준다.
        X_train, X_test = X_df.values[train_index], X_df.values[test_index]
        y_train, y_test = y_df.values[train_index], y_df.values[test_index]
        
        clf.fit(X_train, y_train)                    # 반복학습 ( 이 작업들을 각 fold마다 진행 )
        predictions = clf.predict(X_test)            # 예측값 삽입 
        accuracy = accuracy_score(y_test, predictions) # 각 폴드마다 한 것을 append 해준다.
        scores.append(accuracy)                      # 5개에 대한 정확도를 출력
        print('교차 검증 {0}정확도: {1:.4f}'.format(iter_count, accuracy))
        # print('검증 세트 인덱스: {1}'.format(iter_count, test_index))
        
    mean_score = np.mean(scores)                   # mean_score로 평균을 구해준다.(5개 결과를 저장한 scores를 호출하여)
    print('평균 정확도: {0: .4f}'.format(mean_score))
        
exec_kfold(rf_clf, folds=5) 

교차 검증 0정확도: 0.7710
교차 검증 1정확도: 0.8473
교차 검증 2정확도: 0.7405
교차 검증 3정확도: 0.6947
교차 검증 4정확도: 0.7816
평균 정확도:  0.7670


In [14]:
# GridSearchCV : 파라미터를 통해 성능을 튜닝. (타이타닉 생존률을 해결하기 위해 필요한 작업)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from xgboost import XGBClassifier

parameters = {'n_estimators':[50,100,200],'max_depth':[2,3,5],'min_samples_leaf':[1,5,8]}

# x_model = XGBClassifier()
# x_model.fit(X_train, y_train)
# x_pred = x_model.predict(X_test)

# parameters={'booster' :['gbtree'],
#                  'silent':[True],
#                  'max_depth':[5,6,8],
#                  'min_child_weight':[1,3,5],
#                  'gamma':[0,1,2,3],
#                  'nthread':[4],
#                  'colsample_bytree':[0.5,0.8],
#                  'colsample_bylevel':[0.9],
#                  'n_estimators':[50],
#                  'objective':['binary:logistic'],
#                  'random_state':[2]}

cv = KFold(n_splits=6, random_state=1)

grid_dclf = GridSearchCV(rf_clf, param_grid=parameters, scoring='accuracy', cv=5, refit=True) 

display(grid_dclf)
grid_dclf.fit(X_train.values, y_train.values) 

print('GridSearchCV 최적 하이퍼 파라미터:', grid_dclf.best_params_) 

print('GridSearchCV 최고 정확도:', grid_dclf.best_score_) 

# final_params{'booster': ['gbtree'], 'colsample_bylevel': [0.9], 'colsample_bytree': [0.8], 'gamma': [0], 'max_depth': [5], 'min_child_weight': [3], 'n_estimators': [50], 'nthread': [4], 'objective': ['binary:logistic'], 'random_state': [2], 'silent': [True]}


best_dclf = grid_dclf.best_estimator_
display(best_dclf)
dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print('GSCV 예측 정확도 : ', accuracy)

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_eval(y_test, s_pred):
    confusion = confusion_matrix(y_test, s_pred)
    accuracy = accuracy_score(y_test, s_pred)
    precision = precision_score(y_test, s_pred)
    recall = recall_score(y_test, s_pred)
    f1 = f1_score(y_test, s_pred)
    print('오차 행렬')
    print(confusion)
    print()
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

get_clf_eval(y_test, dpredictions)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=11,
                                  

GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 5, 'min_samples_leaf': 1, 'n_estimators': 200}
GridSearchCV 최고 정확도: 0.8042105263157895


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=11, verbose=0,
                       warm_start=False)

GSCV 예측 정확도 :  0.7977099236641222
오차 행렬
[[147   9]
 [ 44  62]]

정확도: 0.7977, 정밀도: 0.8732, 재현율: 0.5849, F1: 0.7006
